___
<a href='https://cafe.naver.com/jmhonglab'><p style="text-align:center;"><img src='https://lh3.googleusercontent.com/lY3ySXooSmwsq5r-mRi7uiypbo0Vez6pmNoQxMFhl9fmZJkRHu5lO2vo7se_0YOzgmDyJif9fi4_z0o3ZFdwd8NVSWG6Ea80uWaf3pOHpR4GHGDV7kaFeuHR3yAjIJjDgfXMxsvw=w2400'  class="center" width="50%" height="50%"/></p></a>
___
<center><em>Content Copyright by HongLab, Inc.</em></center>

# 음성 비서 업그레이드

### 목표
- 도시 이름이 들어 있으면 그 도시의 날짜/시간/날씨 안내
- 함수를 사용해서 코드의 구조를 개선


### 접근 방법
- 창의적인 아이디어를 덧붙이는 것이 최고  
- 정답을 찾으려고 하지 말고 내가 상상하는 것을 만들려고 노력  
- 좋은 구조의 기준은 사람에 따라 다르지만 일반적으로 이해하기 쉽고 기능 개선과 관리가 유리

### 세계 시간

##### 파이썬 TimeZone [pytz](http://pytz.sourceforge.net/) 패키지

```
pip install pytz
```
아래 예제 참고



In [1]:
# 파이썬 timezone 사용 예시

from datetime import datetime, timedelta
from pytz import timezone

tz = timezone("Asia/Seoul")
current_time = datetime.now().astimezone(tz)

print(current_time)

2024-09-02 12:30:27.556364+09:00


```timezone()```을 설정할 때 알아야 하는 ```zone``` 문자열(위의 예제에서는 "Asia/Seoul")은 ```all_timezones```에서 찾을 수 있습니다.

In [1]:
# 지원하는 타임존을 모두 출력해볼 수도 있지만 너무 많습니다.

from pytz import all_timezones

all_timezones

['Africa/Abidjan', 'Africa/Accra', 'Africa/Addis_Ababa', 'Africa/Algiers', 'Africa/Asmara', 'Africa/Asmera', 'Africa/Bamako', 'Africa/Bangui', 'Africa/Banjul', 'Africa/Bissau', 'Africa/Blantyre', 'Africa/Brazzaville', 'Africa/Bujumbura', 'Africa/Cairo', 'Africa/Casablanca', 'Africa/Ceuta', 'Africa/Conakry', 'Africa/Dakar', 'Africa/Dar_es_Salaam', 'Africa/Djibouti', 'Africa/Douala', 'Africa/El_Aaiun', 'Africa/Freetown', 'Africa/Gaborone', 'Africa/Harare', 'Africa/Johannesburg', 'Africa/Juba', 'Africa/Kampala', 'Africa/Khartoum', 'Africa/Kigali', 'Africa/Kinshasa', 'Africa/Lagos', 'Africa/Libreville', 'Africa/Lome', 'Africa/Luanda', 'Africa/Lubumbashi', 'Africa/Lusaka', 'Africa/Malabo', 'Africa/Maputo', 'Africa/Maseru', 'Africa/Mbabane', 'Africa/Mogadishu', 'Africa/Monrovia', 'Africa/Nairobi', 'Africa/Ndjamena', 'Africa/Niamey', 'Africa/Nouakchott', 'Africa/Ouagadougou', 'Africa/Porto-Novo', 'Africa/Sao_Tome', 'Africa/Timbuktu', 'Africa/Tripoli', 'Africa/Tunis', 'Africa/Windhoek', 'Ameri

In [2]:
# 내가 찾는 도시가 들어있는지 확인해볼 수 있습니다.

from pytz import all_timezones

[z for z in all_timezones if "seoul" in z.lower()]

['Asia/Seoul']

몇 개 미리 찾아놨어요.
```
'Asia/Seoul', 'America/New_York', 'America/Los_Angeles', 'Europe/Paris', 'Europe/London'
```

### 세계 날씨


[OpenWeather에서 도시 영어 이름 확인](https://openweathermap.org/find)


In [5]:
# 앞에서 공부한 OpenWeather API 사용법입니다.
import requests

API_KEY = "4a9a2de86defcf6415366cf6a19d2250"
BASE_URL = "http://api.openweathermap.org/data/2.5/weather"
LANGUAGE = "kr"

city = "seoul"  # 여기에 위에서 찾은 도시 이름 사용
request_url = f"{BASE_URL}?appid={API_KEY}&q={city}&lang={LANGUAGE}"

# 이하 생략

'http://api.openweathermap.org/data/2.5/weather?appid=4a9a2de86defcf6415366cf6a19d2250&q=seoul&lang=kr'

### 사용자 명령에서 도시 이름 찾기

인간의 언어를 컴퓨터로 분석하고 이해하는 것은 어려운 일입니다. 전문적인 자연어 처리에는 파이썬의 [NLTK](https://www.nltk.org/)를 많이 사용합니다.

여기서는 간단히 사용자의 말에 도시 이름이 들어있는지 찾아주는 함수를 만들어서 사용하겠습니다.

**작동 예시**
```
find_keyword(["서울", "뉴욕"], "뉴욕 시간", "서울") -> "뉴욕" 반환
find_keyword(["서울", "뉴욕"], "파리 날씨", "서울") -> "서울" 반환
```

In [10]:
from datetime import datetime, timedelta
from pytz import timezone

def find_keyword(keywords: list[str], sentence: str, default: str = "") -> str:
    """keywords의 아이템들 중에서 sentence에서 가장 먼저 발견되는 문자열을 반환

    Args:
        keywords (list[str]): 찾고자 하는 키워드의 리스트 예) ["서울", "뉴욕"]
        sentence (str): 키워드를 찾아볼 문장 예) "뉴욕의 날씨를 알려주세요"
        default (str): 키워드가 하나도 없을 경우 반환할 문자열 예) "서울"

    Returns:
        str: 문장 안에서 처음 발견한 키워드. 없을 경우 default 반환.
    """

    # 여기에 구현
    # 방법1 : for문 사용
    # for keyword in keywords:
    #     if keyword in sentence:
    #         return keyword

    # return default
    
    # 방법2 리스트 컴프리헨션 사용
    found = [keyword for keyword in keywords if keyword in sentence]
    return found[0] if found else default




cities_dict = {
    "서울": "Asia/Seoul",
    "뉴욕": "America/New_York",
    "로스앤젤레스": "America/Los_Angeles",
    "파리": "Europe/Paris",
    "런던": "Europe/London",
}

user_command = "로스앤젤레스의 시간을 알려주세요"  # 또는 더 간단히 "뉴욕 시간"

city = find_keyword(cities_dict.keys(), user_command, "서울")

# tz = timezone("NotCityName") zone이 정확하지 않으면 오류발생
tz = timezone(cities_dict[city])

today = datetime.today().astimezone(tz)

print(city, today)

로스앤젤레스 2024-09-01 20:47:47.471849-07:00


### 실습 방법

1. 앞에서 만들었던 코드를 고쳐서 기능 추가
1. 첨부된 .py 파일을 참고해서 정리 (스크립트 모드 권장)
